#**Tesi Mattia Polticchia**

---



---



Bug e problemi
---
Abbiamo dei problemi di: 
`[Tensorflow 2.0] AttributeError: Tensor.op is meaningless when eager execution is enabled`
Una possibile soluzione è questo `train_interval=4` messo nella creazione dell' DQNAgent.

Possibile step, creare una funzione che crei dei grafici coi dati dei vari train. Potrebbe essere interessante da mostrare nella tesi.

---

A quanto pare c'è una montagna di roba nelle commit di github che va accuratamente osservata.

Modifiche nel tempo
---
* Cambiata la policy dopo una selezione fatta seguendo un articolo su un sito 
specialistico. Ritrovare il sito ( controllare telegram )

* Rimossi alcuni parametri nella creazione del DQNAgent perché superflui in quanto già presenti nella dichiarazione dalla classe. Ha senso metterli solo  se si vogliono diversi valori rispetto agli standard.

---
##Rete neurale
* Aggiunto il valore di dropout per evitare l'overfitting. Questo ha effettivamente portato a dei miglioramenti in tempo e in quantità di punti.
>_Paper di 30 pagine da leggere - Inserire il titolo_
* Modificata la struttura dei nodi e la dichiarazione dell'attivatore \\
from :
```
agent.add(Dense(16))
agent.add(Activation('relu'))
```
to : \\
 `model.add(Dense(16, activation='relu'))`


---
##Link vari
* Creare un piccolo file log delle modifiche della gym-tetris che sta subendo sporadiche modifiche nel tempo ma non vengono segnalate quasi da nessuna parte.
>tetris_env i return. \\
>il dict che era nelle info è stato commentato -> spiegare perché

* Aggiungere da qualche parte tutta la miriade di link che sono su telegram in modo da averli pronti per la bibliografia e le note


# Codice

In [0]:
!pip install keras==2.3.1
!pip install keras-rl
!pip install gym-tetris
!pip install nes-py
!pip install tensorflow-gpu==2.0.0-beta1
!pip install numpy==1.16.4

In [0]:
'''
Print the path to file for modification of some stuff
make # to row 186
next update, edit the "NoneType" error.
Probably the error is given when the env stop and return the next piece
what a weird stuff
'''
import gym_tetris as gym
print(gym.__path__)
#Check if GPU is used
import tensorflow as tf
tf.test.gpu_device_name()

from google.colab import drive
drive.mount('/content/drive')


In [0]:
import sys
import numpy as no

from nes_py.wrappers import JoypadSpace
import gym_tetris
from gym_tetris.actions import MOVEMENT, SIMPLE_MOVEMENT

from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy, LinearAnnealedPolicy, BoltzmannQPolicy, BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory
from rl.callbacks import FileLogger, ModelIntervalCheckpoint
env = gym_tetris.make('TetrisA-v1')
env = JoypadSpace(env, SIMPLE_MOVEMENT)


#this can be the output range 0-6 for the move
num_actions = env.action_space.n
input_shape = (1,) + env.observation_space.shape

print("Num actions: ", num_actions)

#Build the NN
model = Sequential()
model.add(Flatten(input_shape=input_shape))
model.add(Dense(32, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(num_actions, activation='linear'))
print(model.summary())


Using TensorFlow backend.


Num actions:  5
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 184320)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                5898272   
_________________________________________________________________
dense_2 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_3 (Dense)              (None, 512)               33280     
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 2565      
Total params: 5,936,229
Trainable params: 5,936,229
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
policy = BoltzmannGumbelQPolicy()
memory= SequentialMemory(limit=50000, window_length = 1)
dqn = DQNAgent(model = model, nb_actions = num_actions , memory=memory, train_interval=4,
               nb_steps_warmup=5000, target_model_update=10000, policy=policy)

dqn.compile(Adam(lr=1e-4), metrics=['mae'])
dqn.fit(env, nb_steps=500000, visualize=False, log_interval=50000)

weights_filename = '/content/drive/My Drive/Colab Notebooks/dqn_{}_weights.h5f'.format("gym_tetris")
dqn.save_weights(weights_filename, overwrite=True)

In [0]:
#@title Reopen
#reopen the fit function but i'm not quite sure for that
weights_filename= '/content/dqn_{}_weights.h5f'.format('gym_tetris')
dqn.load_weights(weights_filename)
dqn.fit(env, nb_steps=500000, visualize=False, log_interval=50000)
dqn.save_weights(weights_filename, overwrite=True)


In [0]:
#@title Reload 
#Reload the weight and make test

policy = BoltzmannQPolicy()
memory= SequentialMemory(limit=50000, window_length = 1)

weights_filename= '/content/dqn_{}_weights.h5f'.format('gym_tetris')
dqn.load_weights(weights_filename)
nb_episodes = 5 

dqn.test(env,nb_episodes=nb_episodes,visualize=False)